## Decorators

* decorators allow to factor out certain cross-cutting concerns
* they are used as an integration tool (e.g. joining routes of a web application with handlers)
* they work, because python has first class functions

Examples:

```python
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'
```

* more examples: [https://wiki.python.org/moin/PythonDecoratorLibrary](https://wiki.python.org/moin/PythonDecoratorLibrary)

### Writing a decorator

A decorator is syntactic sugar for:

```python

@decorator
def some_function():
    pass

some_function = decorator(some_function)
```

The main idea is to return a function, that wraps the original function.

* [Snippets/Decorator](Snippets/Decorator)

```python
def deco(f):
    def inner(*args, **kwargs):
        print("[deco] calling {}".format(f.__name__))
        result = f(*args, **kwargs)
        print("[deco] exited {}".format(f.__name__))
        return result
    return inner

@deco
def hello(name="world"):
    print("hello " + name)

hello()
```


## Decorators with and without arguments

Decorators can take arguments or not.

```python
@deco
def f():
    pass
    
@deco(a=1)
def f():
    pass
```


## Decorators can be stacked

In [3]:
def deco1(f):
    def inner(*args, **kwargs):
        print("deco1")
        f(*args, **kwargs)
    return inner

def deco2(f):
    def inner(*args, **kwargs):
        print("deco2")
        f(*args, **kwargs)
    return inner


In [4]:
@deco1
@deco2
def f():
    print("f")

In [5]:
f()

deco1
deco2
f


Task: write small decorator that caches return values of functions

In [13]:
def cache(f):
    cache = {}
    def inner(*args, **kwargs):
        key = f"{args}{kwargs}"
        try:
            return cache[key]
        except KeyError:
            print("not cached")
            result = f(*args, **kwargs)
            cache[key] = result
        return cache[key]
    return inner

In [16]:
@cache
def hello(name):
    return f"hello {name}"

In [17]:
hello("world")

not cached


'hello world'

In [18]:
hello("world")

'hello world'

In [19]:
hello("martin")

not cached


'hello martin'

In [20]:
hello("martin")

'hello martin'

## Functools Helper

* we can wrap the inner function (with a decorator) to keep name and docstring

```python
from functools import wraps


def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

example()

print(example.__name__)
print(example.__doc__)
```

Decorators in the standard library

* functools.wraps
* functools.cache (3.9)
* functools.cached_property
* functools.lru_cache (128)
* functools.singledispatch (https://peps.python.org/pep-0443/, Single-dispatch generic functions)

In [13]:
import functools, statistics

In [15]:
class DataSet:

    def __init__(self, sequence_of_numbers):
        self._data = tuple(sequence_of_numbers)

    @functools.cached_property
    def stdev(self):
        return statistics.stdev(self._data)


In [19]:
ds = DataSet(range(1000))

In [20]:
%time ds.stdev

CPU times: user 0 ns, sys: 5.7 ms, total: 5.7 ms
Wall time: 4.04 ms


288.8194360957494

In [21]:
%time ds.stdev

CPU times: user 16 µs, sys: 1 µs, total: 17 µs
Wall time: 28.8 µs


288.8194360957494

In [6]:
@functools.lru_cache
def count_vowels(sentence):
    return sum(sentence.count(vowel) for vowel in 'AEIOUaeiou')

In [11]:
%time count_vowels("abcde" * 10000)

CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.56 ms


20000

In [12]:
%time count_vowels("abcde" * 10000)

CPU times: user 91 µs, sys: 3 µs, total: 94 µs
Wall time: 107 µs


20000

In [25]:
from functools import singledispatch

@singledispatch
def add(a, b):
    raise NotImplementedError('Unsupported type')

@add.register(int)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)

@add.register(str)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)

@add.register(list)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)

if __name__ == '__main__':
    add(1, 2)
    add('Python', 'Programming')
    add([1, 2, 3], [5, 6, 7])

First argument is of type  <class 'int'>
3
First argument is of type  <class 'str'>
PythonProgramming
First argument is of type  <class 'list'>
[1, 2, 3, 5, 6, 7]


### Decorators with constructors

In [1]:

import time


def retry(count=3, sleep=0):
    """
    @retry():
    def some_func():
        pass
    @retry(count=2, sleep=1):
    def some_func():
        pass
    """

    # The decorator
    def __decorator(func):
        
        # The retry function
        def __retry_func(*args, **kwargs):
            for _ in range(count):
                try:
                    return func(*args, **kwargs)
                except Exception as exc:
                    time.sleep(sleep)
                    last_exc = exc
            raise last_exc
        return __retry_func
    return __decorator

In [6]:
import random

In [22]:
@retry(count=1)
def f():
    if random.random() > 0.5:
        print("err")
        raise RuntimeError()
    print("ok")

In [23]:
f()

err


RuntimeError: 